In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
import numpy as np
import requests
import pandas as pd
import datetime

In [11]:
headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

In [4]:
x = 5

In [72]:
%%time
def get_data(x):
    total_data = []
    for i in [str(i+1) for i in range(x)]:
        link = 'https://bbs.pku.edu.cn/v2/thread.php?bid=896&mode=topic&page='+i
        res = requests.get(link,headers = headers)
        total_data.append(res.text)
    total_data = '.'.join(total_data)
    total_list = re.findall('<!-- list item -->(.*?)<div class="time">(.*?)</div>\n',total_data,re.S)
    result = [] 
    for i in range(len(total_list)):
        total_list[i] = list(total_list[i])
        sub_result = []
        sub_result.append(re.sub('&nbsp;','',re.findall('<div class="title l limit" style="max-width: 4.*?px;">(.*?)</div>',total_list[i][0],re.S)[0]))
        link = 'https://bbs.pku.edu.cn/v2/'+re.sub('amp;','',re.findall('<div class="list-item-topic list-item" data-itemid=".*?"><a class="link" href="(.*?)"></a>',total_list[i][0],re.S)[0])
        sub_result.append(link)
        sub_result.append(total_list[i][1])
        result.append(sub_result)
    data=pd.DataFrame(result,columns = ['实习名称','访问链接','时间'])
    def trans(x):
        res = requests.get(x,headers = headers)
        reg_exp = '1楼 \n\n\n            \n            </span>\n\n            <div class="content">\n              <div class="body file-read image-click-view">(.*?)</div>\n            </div>\n            <div class="operations">'
        JD = re.findall(reg_exp,res.text,re.S)[0]
        JD = re.sub('&nbsp;','',JD)
        JD = re.sub('<.*?>','',JD)
        return JD
    data['JD'] = data['访问链接'].transform(trans)
    data['时间'] = data['时间'].apply(lambda x : datetime.date.today().strftime('%m-%d')+' ' + x if len(x) < 11 else x)
    return data


CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 21 µs
